In [0]:
#!pip install pycaret

#1. Getting Data
In this step-by-step tutorial, we will use ‘diabetes’ dataset and the goal is to predict patient outcome (binary 1 or 0) based on several factors such as Blood Pressure, Insulin Level, Age etc. The dataset is available on PyCaret’s github repository. Easiest way to import dataset directly from repository is by using get_data function from pycaret.datasets modules.

In [0]:
from pycaret.datasets import get_data
diabetes = get_data('diabetes')

#2. Setting up Environment
The first step of any machine learning experiment in PyCaret is setting up the environment by importing the required module and initializing setup( ). The module used in this example is pycaret.classification.
Once the module is imported, setup() is initialized by defining the dataframe (‘diabetes’) and the target variable (‘Class variable’).

In [0]:
from pycaret.classification import *
exp1 = setup(diabetes, target = 'Class variable')

All the preprocessing steps are applied within setup(). With over 20 features to prepare data for machine learning, PyCaret creates a transformation pipeline based on the parameters defined in setup function. It automatically orchestrates all dependencies in a pipeline so that you don’t have to manually manage the sequential execution of transformations on test or unseen dataset. PyCaret’s pipeline can easily be transferred across environments to run at scale or be deployed in production with ease. Below are preprocessing features available in PyCaret as of its first release.
#image link:
https://miro.medium.com/max/1400/1*jo9vPsQhQZmyXUhnrt9akQ.png

💡 Data Preprocessing steps that are compulsory for machine learning such as missing values imputation, categorical variable encoding, label encoding (converting yes or no into 1 or 0), and train-test-split are automatically performed when setup() is initialized. Click here https://pycaret.org/preprocessing/ to learn more about PyCaret’s preprocessing abilities.

#3. Compare Models
This is the first step recommended in supervised machine learning experiments (classification or regression). This function trains all the models in the model library and compares the common evaluation metrics using k-fold cross validation (by default 10 folds). The evaluation metrics used are:
For Classification: Accuracy, AUC, Recall, Precision, F1, Kappa
For Regression: MAE, MSE, RMSE, R2, RMSLE, MAPE

In [0]:
compare_models()

💡 Metrics are evaluated using 10-fold cross validation by default. It can be changed by changing the value of fold parameter.
💡 Table is sorted by ‘Accuracy’ (Highest to Lowest) value by default. It can be changed by changing the value of sort parameter.

#4. Create Model
Creating a model in any module of PyCaret is as simple as writing create_model. It takes only one parameter i.e. the model name passed as string input. This function returns a table with k-fold cross validated scores and a trained model object.

In [0]:
adaboost = create_model('ada')

Variable ‘adaboost’ stores a trained model object returned by create_model function is a scikit-learn estimator. Original attributes of a trained object can be accessed by using period ( . ) after variable. See example below.

💡 PyCaret has over 60 open source ready-to-use algorithms. Click here https://pycaret.org/create-model/ to see a complete list of estimators / models available in PyCaret.

#5. Tune Model
The tune_model function is used for automatically tuning hyperparameters of a machine learning model. PyCaret uses random grid search over a predefined search space. This function returns a table with k-fold cross validated scores and a trained model object.

In [0]:
tuned_adaboost = tune_model('ada')

💡 The tune_model function in unsupervised modules such as pycaret.nlp, pycaret.clustering and pycaret.anomaly can be used in conjunction with supervised modules. For example, PyCaret’s NLP module can be used to tune number of topics parameter by evaluating an objective / cost function from a supervised ML model such as ‘Accuracy’ or ‘R2’.

#6. Ensemble Model
The ensemble_model function is used for ensembling trained models. It takes only one parameter i.e. a trained model object. This functions returns a table with k-fold cross validated scores and a trained model object.

In [0]:
# creating a decision tree model
dt = create_model('dt')
# ensembling a trained dt model
dt_bagged = ensemble_model(dt)

💡 ‘Bagging’ method is used for ensembling by default which can be changed to ‘Boosting’ by using the method parameter within the ensemble_model function.
💡 PyCaret also provide blend_models and stack_models functionality to ensemble multiple trained models.

In [0]:
# blending all models
blend_all = blend_models()

In [0]:
from pycaret.classification import *
# create individual models for stacking
ridge = create_model('ridge')
lda = create_model('lda')
gbc = create_model('gbc')
xgboost = create_model('xgboost')
# stacking models
stacker = stack_models(estimator_list = [ridge,lda,gbc], meta_model = xgboost)

#7. Plot Model
Performance evaluation and diagnostics of a trained machine learning model can be done using the plot_model function. It takes a trained model object and the type of plot as a string input within the plot_model function.

In [0]:
# create a model
adaboost = create_model('ada')
# AUC plot
plot_model(adaboost, plot = 'auc')
# Decision Boundary
plot_model(adaboost, plot = 'boundary')
# Precision Recall Curve
plot_model(adaboost, plot = 'pr')
# Validation Curve
plot_model(adaboost, plot = 'vc')

Click here https://pycaret.org/plot-model/ to learn more about different visualization in PyCaret.
Alternatively, you can use evaluate_model function to see plots via user interface within notebook.

In [0]:
evaluate_model(adaboost)

💡 plot_model function in pycaret.nlp module can be used to visualize text corpus and semantic topic models. Click here to learn more about it.

#8. Interpret Model
When the relationship in data is non-linear which is often the case in real life we invariably see tree-based models doing much better than simple gaussian models. However, this comes at the cost of losing interpretability as tree-based models do not provide simple coefficients like linear models. PyCaret implements SHAP (SHapley Additive exPlanations using interpret_model function.

In [0]:
# create a model
xgboost = create_model('xgboost')
# summary plot
interpret_model(xgboost)


In [0]:
# correlation plot
interpret_model(xgboost, plot = 'correlation')

Interpretation of a particular datapoint (also known as reason argument) in the test dataset can be evaluated using ‘reason’ plot. In the below example we are checking the first instance in our test dataset.

In [0]:
interpret_model(xgboost, plot = 'reason', observation = 0) 

#9. Predict Model
So far the results we have seen are based on k-fold cross validation on training dataset only (70% by default). In order to see the predictions and performance of the model on the test / hold-out dataset, the predict_model function is used.

In [0]:
# create a model
rf = create_model('rf')


In [0]:
# predict test / hold-out dataset
rf_holdout_pred = predict_model(rf)

predict_model function is also used to predict unseen dataset. For now, we will use the same dataset we have used for training as a proxy for new unseen dataset. In practice, predict_model function would be used iteratively, every time with a new unseen dataset.

In [0]:
predictions = predict_model(rf, data = diabetes)
predictions

💡 predict_model function can also predict a sequential chain of models which are created using stack_models and create_stacknet function.
💡 predict_model function can also predict directly from the model hosted on AWS S3 using deploy_model function.

#10. Deploy Model
One way to utilize the trained models to generate predictions on an unseen dataset is by using the predict_model function in the same notebooks / IDE in which model was trained. However, making the prediction on an unseen dataset is an iterative process; depending on the use-case, the frequency of making predictions could be from real time predictions to batch predictions. PyCaret’s deploy_model function allows deploying the entire pipeline including trained model on cloud from notebook environment.

In [0]:
deploy_model(model = rf, model_name = 'rf_aws', platform = 'aws', 
             authentication =  {'bucket'  : 'pycaret-test'})

#11. Save Model / Save Experiment
Once training is completed the entire pipeline containing all preprocessing transformations and trained model object can be saved as a binary pickle file.

In [0]:
# creating model
adaboost = create_model('ada')
# saving model
save_model(adaboost, model_name = 'ada_for_deployment')

We can also save the entire experiment consisting of all intermediary outputs as one binary file.

In [0]:
save_experiment(experiment_name = 'my_first_experiment')

💡 We can load saved model and saved experiment using load_model and load_experiment function available in all modules of PyCaret.